Aadhar Dataset -- Merged dataset



## DATA CLEANING

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()


In [ ]:
# loading the dataset
df =  pd.read_csv('/content/merged_aadhar_enrolment.csv',
    encoding="latin1",
    low_memory=False
)

df.head()

In [ ]:
# shape  of the data frame
df.shape

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
"""In our data Frame age group 5 is repeating then we have to make it clear
  where to put 5 years olg group"""
df = df.rename(columns={
    'age_0_5': 'age_0_4',
    'age_5_17': 'age_5_17',
    'age_18_greater': 'age_18_plus'
})
df.columns

In [ ]:
# Converting  data types:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

age_cols = ['age_0_4', 'age_5_17', 'age_18_plus']
for col in age_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df['pincode'] = pd.to_numeric(df['pincode'], errors='coerce')

df.info()

In [ ]:
# checking null values:
print(df.isnull().sum())
print("No null values found")

In [ ]:
# Checking for the duplicate rows..
df.duplicated().sum()


Number of duplicated rows = 22,957


In [ ]:
# Removing duplicate rows:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

In [ ]:
# Remove Invalid Rows:
df = df.dropna(subset=['date', 'state', 'district', 'pincode'])


In [ ]:
# Checking if negative values or invalid values are present
df = df[
    (df['age_0_4'] >= 0) &
    (df['age_5_17'] >= 0) &
    (df['age_18_plus'] >= 0)
]


In [ ]:
# Normalize Text Fields
df['state'] = df['state'].str.title().str.strip()
df['district'] = df['district'].str.title().str.strip()


In [ ]:
# Validate Pincode
df = df[(df['pincode'] >= 100000) & (df['pincode'] <= 999999)]


In [ ]:
# unique count of state
df['state'].nunique()

In [ ]:
df['state'].unique()

In [ ]:
# Insight: Since we have multiple states with different names
#          we need to make them correct.
import pandas as pd
import re

def clean_state_name(x):
    if pd.isna(x):
        return x
    x = str(x).lower()
    x = re.sub(r'[^a-z\s]', ' ', x)   # remove symbols like &, .
    x = re.sub(r'\s+', ' ', x).strip()  # remove extra spaces
    return x



In [ ]:
state_mapping = {
    # West Bengal
    'west bengal': 'West Bengal',
    'west bangal': 'West Bengal',
    'westbengal': 'West Bengal',
    'west  bengal': 'West Bengal',
    'westbengal ': 'West Bengal',

    # Andhra Pradesh
    'andhra pradesh': 'Andhra Pradesh',

    # Odisha
    'odisha': 'Odisha',
    'orissa': 'Odisha',

    # Jammu & Kashmir
    'jammu and kashmir': 'Jammu and Kashmir',
    'jammu kashmir': 'Jammu and Kashmir',

    # Collapse all Dadra / Daman variants into ONE state
    'dadra nagar haveli': 'Dadra and Nagar Haveli and Daman and Diu',
    'dadra and nagar haveli': 'Dadra and Nagar Haveli and Daman and Diu',
    'daman and diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'dadra and nagar haveli and daman and diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'the dadra and nagar haveli and daman and diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'daman diu': 'Dadra and Nagar Haveli and Daman and Diu',


    # Puducherry
    'puducherry': 'Puducherry',


    # Andaman & Nicobar Islands
    'andaman and nicobar islands': 'Andaman and Nicobar Islands',
    'andaman nicobar islands': 'Andaman and Nicobar Islands',

    # Puducherry
    'pondicherry': 'Puducherry',

    # Case-normalized direct matches
    'delhi': 'Delhi',
    'ladakh': 'Ladakh',
    'goa': 'Goa',
    'sikkim': 'Sikkim',
    'assam': 'Assam',
    'bihar': 'Bihar',
    'punjab': 'Punjab',
    'kerala': 'Kerala',
    'haryana': 'Haryana',
    'gujarat': 'Gujarat',
    'tamil nadu': 'Tamil Nadu',
    'telangana': 'Telangana',
    'karnataka': 'Karnataka',
    'maharashtra': 'Maharashtra',
    'rajasthan': 'Rajasthan',
    'uttar pradesh': 'Uttar Pradesh',
    'madhya pradesh': 'Madhya Pradesh',
    'himachal pradesh': 'Himachal Pradesh',
    'arunachal pradesh': 'Arunachal Pradesh',
    'chhattisgarh': 'Chhattisgarh',
    'jharkhand': 'Jharkhand',
    'manipur': 'Manipur',
    'meghalaya': 'Meghalaya',
    'mizoram': 'Mizoram',
    'nagaland': 'Nagaland',
    'tripura': 'Tripura',
    'uttarakhand': 'Uttarakhand',
    'lakshadweep': 'Lakshadweep',
    'chandigarh': 'Chandigarh'
}


In [ ]:
df['state_clean'] = (
    df['state']
    .apply(clean_state_name)
    .map(state_mapping)
)


In [ ]:
# Drop invalid entries like numeric junk
df = df[~df['state'].astype(str).str.isnumeric()]

# Optional: check unmapped states
unmapped = df[df['state_clean'].isna()]['state'].unique()
print("Unmapped values:", unmapped)


In [ ]:
df['state_clean'].unique()

In [ ]:
df['state_clean'].nunique()

In [ ]:
df.head(2)

In [ ]:
# Replacing old states to new
df = df.drop(columns=['state'])

df = df.rename(columns={'state_clean': 'state'})


In [ ]:
# taking state column to its originaal place:
cols = df.columns.tolist()
cols.remove('state')
cols.insert(1, 'state')

df = df[cols]

df.head()


In [ ]:
# checking the shape after cleaning:
print(df.shape)
print(df.columns)

In [ ]:
# sorting the dataframe based on state name:
df = df.sort_values(by='state')
df = df.reset_index(drop=True)

df.head(2)

In [ ]:
df.to_csv("aadhaar_cleaned.csv", index=False)

from google.colab import files
files.download("aadhaar_cleaned.csv")
print("Cleaned and sorted Enrollment Dataset has been exported")

## DATA PRE-PROCESSING

In [ ]:
# Loading the dataset:
df = pd.read_csv('/content/aadhaar_cleaned.csv',
    encoding="latin1",
    low_memory=False
)
df.head(2)

In [ ]:
df['state'].nunique()

In [ ]:
# Total Enrolments:
df['total_enrolments'] = (
    df['age_0_4'] +
    df['age_5_17'] +
    df['age_18_plus']
)


In [ ]:
df = df[df['total_enrolments'] > 0]
df.head(2)

In [ ]:
# Age Share Features
df['share_0_4'] = df['age_0_4'] / df['total_enrolments']
df['share_5_17'] = df['age_5_17'] / df['total_enrolments']
df['share_18_plus'] = df['age_18_plus'] / df['total_enrolments']

In [ ]:
# Temporal Features:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['month_name'] = df['date'].dt.month_name()


In [ ]:
print("INSIGHT 1:")
print("Average enrolments per record:",
      round(df['total_enrolments'].mean(), 2))

print("Result: Shows baseline enrolment load per reporting unit.")


In [ ]:
# Age Composition Ratios (CRITICAL)
df['child_ratio'] = (
    (df['age_0_4'] + df['age_5_17']) / df['total_enrolments']
)

df['adult_ratio'] = df['age_18_plus'] / df['total_enrolments']


In [ ]:
print("\nINSIGHT 2:")
print("Mean child ratio:", round(df['child_ratio'].mean(), 2))
print("Mean adult ratio:", round(df['adult_ratio'].mean(), 2))

print("Result: Regions with low adult ratio indicate missed Aadhaar coverage.")

In [ ]:
# Child-Heavy Region Flag (Policy Signal)
df['child_heavy_region'] = df['child_ratio'] > 0.60

print("\nINSIGHT 3:")
print("Child-heavy regions (%):",
      round(df['child_heavy_region'].mean()*100, 2))

print("Result:UIDAI can plan school-based enrolment drives here. ")

In [ ]:
# Peak enrollment months:
monthly_trend = df.groupby('month')['total_enrolments'].mean()
print("\nINSIGHT 4:")
print("Peak enrolment month:", monthly_trend.idxmax())

print("Result: Resource planning during peak enrolment month i.e January")


In [ ]:
# District Enrolment Intensity
df['district_avg_enrolment'] = df.groupby(
    ['state', 'district']
)['total_enrolments'].transform('mean')


In [ ]:
print("\nINSIGHT 5:")
print("Top districts by average enrolment:")
print(
    df.groupby(['state','district'])['district_avg_enrolment']
      .mean()
      .sort_values(ascending=False)
      .head(5)
)


In [ ]:
# Enrolment Volatility:
df['district_volatility'] = df.groupby(
    ['state','district']
)['total_enrolments'].transform('std')


In [ ]:
print("\nINSIGHT 6:")
print("High volatility districts (top 5):")
print(
    df.groupby(['state','district'])['district_volatility']
      .mean()
      .sort_values(ascending=False)
      .head(5)
)


In [ ]:
df.head(4)

In [ ]:
df['district'].nunique()